#Introdução à Programação CUDA


## **Exemplos**

#### Crédito do plugin:
Ricardo Ferreira, Michael Canesche, Westerley Carvalho. Universidade Federal de Viçosa - ricardo@ufv.br

Minicurso do [Simpósio de Sistemas Computacionais de Alto Desempenho](http://wscad.sbc.org.br/2020/chamada-minicursos.html) (WSCAD 2020). [**Links para video, texto completo e slides**](https://github.com/lesc-ufv/wscad2020/blob/master/links/readme.md)

OBS: o minicurso está entre 1h43m25s e 3h38m do vídeo. Se ao clicar você visualizar um erro de reprodução, siga adiante e abra diretamente no YouTube.

## **Configurando o Google Lab (colab)**
Executar o comando abaixo para permitir a execução de códigos de maneira mais simplificada.

In [ ]:
!pip install git+https://github.com/lesc-ufv/cad4u.git &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin

In [ ]:
!nvidia-smi

Wed Jan 29 12:19:48 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Execução da aplicação deviceQuery
O deviceQuery mostra as principais características do dispositivo instalado, incluindo a CUDA capability.

In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git
%cd cuda-samples/Samples/1_Utilities/deviceQuery
!make


Cloning into 'cuda-samples'...
remote: Enumerating objects: 19507, done.
remote: Counting objects: 100% (4370/4370), done.
remote: Compressing objects: 100% (752/752), done.
remote: Total 19507 (delta 4059), reused 3618 (delta 3618), pack-reused 15137 (from 2)
Receiving objects: 100% (19507/19507), 133.52 MiB | 23.92 MiB/s, done.
Resolving deltas: 100% (17186/17186), done.
Updating files: 100% (4026/4026), done.
/content/cuda-samples/Samples/1_Utilities/deviceQuery
/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common -m64 --threads 0 --std=c++11 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o deviceQuery.o -c deviceQuery.cpp
/usr/local

In [ ]:
! ./deviceQuery

./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shar

## Hello World 0

In [ ]:
%%gpu

#include <stdio.h>


__global__ void hello()
{
	printf("Oi mundo! De: thread %d - bloco %d\n", threadIdx.x, blockIdx.x);
}

int main(void)
{
	int num_threads = 2;
	int num_blocks = 2;
	hello<<<num_blocks,num_threads>>>();
	cudaDeviceSynchronize();

	return 0;
}

Oi mundo! De: thread 0 - bloco 0
Oi mundo! De: thread 1 - bloco 0
Oi mundo! De: thread 0 - bloco 1
Oi mundo! De: thread 1 - bloco 1



#Hello World

In [ ]:
%%gpu

#include <stdio.h>

__device__ const char *STR = "HELLO WORLD!";
const char STR_LENGTH = 12;

__global__ void hello()
{
 	printf("%c\n", STR[threadIdx.x % STR_LENGTH] );

}

int main(void)
{
	int num_threads = STR_LENGTH;
	int num_blocks = 1;
	hello<<<num_blocks,num_threads>>>();
	cudaDeviceSynchronize();

	return 0;
}

H
E
L
L
O
 
W
O
R
L
D
!



# Vector Add
## Versão 1: exemplo de bloco

In [ ]:
%%gpu

#include <stdio.h>

#define N 16


__global__ void add(int *a, int *b, int *c)
{
	int index = threadIdx.x;
	c[index] = a[index] + b[index];
	printf("c[%d] = %d\n", index, c[index]);
}


int main()
{
  int *a, *b, *c;
	int *d_a, *d_b, *d_c;
	int size = N * sizeof( int );

	/* allocate space for device copies of a, b, c */

	cudaMalloc( (void **) &d_a, size );
	cudaMalloc( (void **) &d_b, size );
	cudaMalloc( (void **) &d_c, size );

	/* allocate space for host copies of a, b, c and setup input values */

	a = (int *)malloc( size );
	b = (int *)malloc( size );
	c = (int *)malloc( size );

	for( int i = 0; i < N; i++ )
	{
		a[i] = b[i] = i;
		c[i] = 0;
	}

	/* copy inputs to device */
	/* fix the parameters needed to copy data to the device */
	cudaMemcpy( d_a, a, size, cudaMemcpyHostToDevice );
	cudaMemcpy( d_b, b, size, cudaMemcpyHostToDevice );

	/* launch the kernel on the GPU */
	/* insert the launch parameters to launch the kernel properly using blocks and threads */
	add<<<1, N >>>( d_a, d_b, d_c );

	/* copy result back to host */
	/* fix the parameters needed to copy data back to the host */
	cudaMemcpy( c, d_c, size, cudaMemcpyDeviceToHost );

	/* clean up */

	free(a);
	free(b);
	free(c);
	cudaFree( d_a );
	cudaFree( d_b );
	cudaFree( d_c );

	return 0;
} /* end main */

c[0] = 0
c[1] = 2
c[2] = 4
c[3] = 6
c[4] = 8
c[5] = 10
c[6] = 12
c[7] = 14
c[8] = 16
c[9] = 18
c[10] = 20
c[11] = 22
c[12] = 24
c[13] = 26
c[14] = 28
c[15] = 30



# Vector Add
## Versão 2

In [ ]:
%%gpu

#include <stdio.h>

__global__ void add(int *a, int *b, int *c, int n)
{
    /* insert code to calculate the index properly using blockIdx.x, blockDim.x, threadIdx.x */
	int index = blockIdx.x * blockDim.x + threadIdx.x;
	if(index < n)
		c[index] = a[index] + b[index];
}

/* experiment with N */
/* how large can it be? */
#define N (2048*2048)
#define THREADS_PER_BLOCK 512

int main()
{
    int *a, *b, *c;
	int *d_a, *d_b, *d_c;
	int size = N * sizeof( int );

	/* allocate space for device copies of a, b, c */

	cudaMalloc( (void **) &d_a, size );
	cudaMalloc( (void **) &d_b, size );
	cudaMalloc( (void **) &d_c, size );

	/* allocate space for host copies of a, b, c and setup input values */

	a = (int *)malloc( size );
	b = (int *)malloc( size );
	c = (int *)malloc( size );

	for( int i = 0; i < N; i++ )
	{
		a[i] = b[i] = i;
		c[i] = 0;
	}

	/* copy inputs to device */
	/* fix the parameters needed to copy data to the device */
	cudaMemcpy( d_a, a, size, cudaMemcpyHostToDevice );
	cudaMemcpy( d_b, b, size, cudaMemcpyHostToDevice );

	/* launch the kernel on the GPU */
	/* insert the launch parameters to launch the kernel properly using blocks and threads */
	add<<< (N + (THREADS_PER_BLOCK-1)) / THREADS_PER_BLOCK, THREADS_PER_BLOCK >>>( d_a, d_b, d_c, N);

	/* copy result back to host */
	/* fix the parameters needed to copy data back to the host */
	cudaMemcpy( c, d_c, size, cudaMemcpyDeviceToHost );


	printf( "c[0] = %d\n",c[0] );
	printf( "c[%d] = %d\n",N-1, c[N-1] );

	/* clean up */

	free(a);
	free(b);
	free(c);
	cudaFree( d_a );
	cudaFree( d_b );
	cudaFree( d_c );

	return 0;
} /* end main */

c[0] = 0
c[4194303] = 8388606



##Vector Add - Memória unificada

In [ ]:
%%gpu

#include <stdio.h>

__global__ void add(int *a, int *b, int *c, int n)
{
    /* insert code to calculate the index properly using blockIdx.x, blockDim.x, threadIdx.x */
	int index = blockIdx.x * blockDim.x + threadIdx.x;
	if (index < n)
		c[index] = a[index] + b[index];
}

/* experiment with N */
/* how large can it be? */
#define N (2048*2048)
#define THREADS_PER_BLOCK 512

int main()
{
	int *d_a, *d_b, *d_c;
	int size = N * sizeof( int );

	/* allocate space for device copies of a, b, c */

	cudaMallocManaged(&d_a, size );
	cudaMallocManaged(&d_b, size );
	cudaMallocManaged(&d_c, size );

	for( int i = 0; i < N; i++ )
	{
		d_a[i] = d_b[i] = i;
		d_c[i] = 0;
	}

	/* launch the kernel on the GPU */
	/* insert the launch parameters to launch the kernel properly using blocks and threads */
	add<<< (N + (THREADS_PER_BLOCK-1)) / THREADS_PER_BLOCK, THREADS_PER_BLOCK >>>( d_a, d_b, d_c, N);

  cudaDeviceSynchronize();

	printf( "d_c[0] = %d\n", d_c[0] );
	printf( "d_c[%d] = %d\n",N-1, d_c[N-1] );

	/* clean up */

	cudaFree( d_a );
	cudaFree( d_b );
	cudaFree( d_c );

	return 0;
} /* end main */

d_c[0] = 0
d_c[4194303] = 8388606



##**1D Stencil**


In [ ]:
%%gpu

#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out)
{
    __shared__ int temp[BLOCK_SIZE + 2 * RADIUS];
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    int lindex = threadIdx.x + RADIUS;

    // Read input elements into shared memory
    temp[lindex] = in[gindex];
    if (threadIdx.x < RADIUS)
    {
        temp[lindex - RADIUS] = in[gindex - RADIUS];
        temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }

    // Make sure all threads get to this point before proceeding!
    __syncthreads();

    // Apply the stencil
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += temp[lindex + offset];

    // Store the result
    out[gindex-RADIUS] = result;
}

int main()
{
  unsigned int i;
  int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
  int *d_in, *d_out;

  // Initialize host data
  for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
    h_in[i] = 1; // With a value of 1 and RADIUS of 3, all output values should be 7

  // Allocate space on the device
  cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
  cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

  // Copy input data to device
  cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

  stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

  cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

  // Verify every out value is 7
  for( i = 0; i < NUM_ELEMENTS; ++i )
    if (h_out[i] != 7)
    {
      printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
      break;
    }

  if (i == NUM_ELEMENTS)
    printf("SUCCESS!\n");

  // Free out memory
  cudaFree(d_in);
  cudaFree(d_out);

  return 0;
}


### Stencil 2D com memória compartilhada

Este exemplo usa um array 2D de entrada preenchido com números predefinidos ou gerados aleatoriamente. Ele aplica o stencil 2D para calcular a média da vizinhança (3x3) e imprime os valores de entrada e saída.

In [ ]:
%%gpu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define RADIUS 1
#define BLOCK_SIZE 8

// Kernel CUDA para aplicar stencil 2D
__global__ void stencil2D_shared(const float *input, float *output, int width, int height) {
    __shared__ float sharedMem[BLOCK_SIZE + 2 * RADIUS][BLOCK_SIZE + 2 * RADIUS];

    // Coordenadas globais
    int globalX = blockIdx.x * BLOCK_SIZE + threadIdx.x;
    int globalY = blockIdx.y * BLOCK_SIZE + threadIdx.y;

    // Coordenadas locais
    int localX = threadIdx.x + RADIUS;
    int localY = threadIdx.y + RADIUS;

    // Carregar os dados principais para a memória compartilhada
    if (globalX < width && globalY < height) {
        sharedMem[localY][localX] = input[globalY * width + globalX];
    } else {
        sharedMem[localY][localX] = 0.0f;
    }

    // Carregar as bordas
    if (threadIdx.x < RADIUS) {
        sharedMem[localY][threadIdx.x] = (globalX >= RADIUS) ? input[globalY * width + (globalX - RADIUS)] : 0.0f;
        sharedMem[localY][threadIdx.x + BLOCK_SIZE + RADIUS] =
            (globalX + BLOCK_SIZE < width) ? input[globalY * width + (globalX + BLOCK_SIZE)] : 0.0f;
    }

    if (threadIdx.y < RADIUS) {
        sharedMem[threadIdx.y][localX] = (globalY >= RADIUS) ? input[(globalY - RADIUS) * width + globalX] : 0.0f;
        sharedMem[threadIdx.y + BLOCK_SIZE + RADIUS][localX] =
            (globalY + BLOCK_SIZE < height) ? input[(globalY + BLOCK_SIZE) * width + globalX] : 0.0f;
    }

    __syncthreads();

    // Aplicar stencil
    if (globalX < width && globalY < height) {
        float sum = 0.0f;
        for (int dy = -RADIUS; dy <= RADIUS; dy++) {
            for (int dx = -RADIUS; dx <= RADIUS; dx++) {
                sum += sharedMem[localY + dy][localX + dx];
            }
        }
        output[globalY * width + globalX] = sum / ((2 * RADIUS + 1) * (2 * RADIUS + 1));
    }
}

void printMatrix(const char *label, float *matrix, int width, int height) {
    printf("%s:\n", label);
    for (int i = 0; i < height; i++) {
        for (int j = 0; j < width; j++) {
            printf("%6.2f ", matrix[i * width + j]);
        }
        printf("\n");
    }
}

int main() {
    // Dimensões da matriz
    int width = 16;
    int height = 16;

    // Tamanho em bytes
    size_t size = width * height * sizeof(float);

    // Alocar e inicializar a matriz no host
    float *h_input = (float *)malloc(size);
    float *h_output = (float *)malloc(size);

    for (int i = 0; i < width * height; i++) {
        h_input[i] = rand() % 10 + 1; // Valores aleatórios entre 1 e 10
    }

    // Alocar memória no dispositivo
    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);

    // Copiar a matriz de entrada para o dispositivo
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    // Configurar dimensões de bloco e grid
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize((width + BLOCK_SIZE - 1) / BLOCK_SIZE, (height + BLOCK_SIZE - 1) / BLOCK_SIZE);

    // Chamar o kernel
    stencil2D_shared<<<gridSize, blockSize>>>(d_input, d_output, width, height);

    // Copiar o resultado de volta para o host
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    // Imprimir matrizes de entrada e saída
    printMatrix("Matriz de entrada", h_input, width, height);
    printMatrix("Matriz de saída (após stencil)", h_output, width, height);

    // Liberar memória
    free(h_input);
    free(h_output);
    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}
